In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
#from k_class_perceptron import k_class_perceptron
from sklearn.model_selection import train_test_split, KFold
from prettytable import PrettyTable
from muti_class_perceptron_1vRest import k_class_perceptron_1vRest
from multi_class_perceptron_1v1 import k_class_perceptron_1v1
#from skmultilearn.adapt import MLkNN
#from kernel_Perceptron_1 import kernel_Perceptron
import cos_knn as knn
import svc as svc

#data = pd.read_table("zipcombo.dat", sep="\s+")
data = pd.read_table("dtrain123.dat", sep="\s+")
data = np.array(data)

In [2]:
def k_fold_crossvalidation(k, data_x, data_y, d, iteration, kernel = "poly"):
	#split data into k segments averagely
	kf=KFold(n_splits=k)
	a = kf.split(data_x)
	mistake_list = []
	for train_data_index, test_data_index in a:
		#split data set
		x_train = data_x[train_data_index]
		x_test = data_x[test_data_index]
		y_train = data_y[train_data_index]
		y_test = data_y[test_data_index]
		#perceptron
		kp = k_class_perceptron_1vRest(len(x_train),x_train,y_train,10,d, kernel)
		kp.train(iteration)
		mistakes = test_classifiers(x_test,y_test,kp)
		mistake_list.append(mistakes)
		print("-------------------")
	sum = 0
	for i in range(len(mistake_list)):
		sum+= mistake_list[i]
	mean_mistake = sum/len(mistake_list)
	return mean_mistake

In [3]:
def test_classifiers(test_data,test_labels,perceptron):
	'''
	mistake_count = 0
	for i in range(len(test_data)):
		confidence, result = perceptron.predict(test_data[i])
		if(result != test_labels[i]):
			mistake_count = mistake_count + 1
	return mistake_count
	'''
	mistake_count = 0
	confident_mtx = perceptron.predict_mtx(test_data)
	mistake_count = 0
	for i in range(len(confident_mtx)):
		indivadual_confident_list = confident_mtx[i].tolist()
		predict_label = indivadual_confident_list.index(max(indivadual_confident_list))
		if(int(predict_label) != int(test_labels[i])):
			mistake_count += 1
	return mistake_count

In [4]:
def part1_6_1():
	iteration = 10
	result_table = PrettyTable(["train errors ± STD","test errors ± STD"])
	train_result_mtx = np.zeros(shape = (7,20))
	test_result_mtx = np.zeros(shape = (7,20))
	for d in range(1,8):
		mistake_train_mtx = np.zeros(shape = (20,iteration))
		mistake_test_mtx = np.zeros(shape = (20,1))
		for i in range(20):
			print("---------------")
			print("d = " + str(d)+", round: "+ str(i))
			#split data set
			data_train ,data_test = train_test_split(data,test_size=0.2)
			x_train = data_train[:,1:]
			y_train = data_train[:,0]
			x_test = data_test[:,1:]
			y_test = data_test[:,0]


			#perceptron
			kp = k_class_perceptron_1v1(len(x_train),x_train,y_train,10,d)
			mistake_train = kp.train(iteration)
			mistake_train_mtx[i] = mistake_train
			mistake_test = test_classifiers(x_test,y_test,kp)
			mistake_test_mtx[i] = mistake_test
			#print(mistake_test_mtx)
		mistake_train_mtx = np.sum(mistake_train_mtx, axis=1)
		mistake_test_mtx = np.sum(mistake_test_mtx, axis=1)
		train_result_mtx[d-1] = mistake_train_mtx
		test_result_mtx[d-1] = mistake_test_mtx
	#print(test_result_mtx)

	for i in range(len(train_result_mtx)):
		mean_train_error = np.sum(train_result_mtx[i])/20
		train_std = np.std(train_result_mtx[i], ddof = 1)
		mean_test_error = np.sum(test_result_mtx[i])/20
		test_std = np.std(test_result_mtx[i], ddof = 1)
		result_table.add_row([str(mean_train_error) + ' ± ' +str(train_std), str(mean_test_error)+ ' ± ' +str(test_std)])
	print(result_table)

In [ ]:
part1_6_1()